In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from binascii import unhexlify
from zlib import crc32
import logging as log

In [ ]:
def encrypt_text(plain_text, encrypt_key):
    key = unhexlify(encrypt_key)
    nonce = b'\0' * len(key)

    cipher = Cipher(algorithms.AES(key), modes.CTR(nonce), backend=default_backend())
    enc = cipher.encryptor()

    # Convert text to bytes
    text_bytes = plain_text.encode('utf-8')

    # Encrypt the text
    enc_block = enc.update(text_bytes)
    crc = crc32(enc_block)
    encrypted_text = enc_block
    return encrypted_text

def decrypt_text(encrypted_text, encrypt_key):
    key = unhexlify(encrypt_key)
    nonce = b'\0' * len(key)

    cipher = Cipher(algorithms.AES(key), modes.CTR(nonce), backend=default_backend())
    dec = cipher.decryptor()

    # Decrypt the encrypted text
    dec_block = dec.update(encrypted_text)
    crc = crc32(dec_block)
    decrypted_text = dec_block.decode('utf-8')
    return decrypted_text

# Example usage
plaintext = "98765432"
encryption_key = "00112233445566778899aabbccddeeff"

# Encrypt
encrypted_text = encrypt_text(plaintext, encryption_key)

# Decrypt
decrypted_text = decrypt_text(encrypted_text, encryption_key)

In [ ]:
print("Original Text:", plaintext)
print("Encrypted Text:", encrypted_text.hex())
print("Decrypted Text:", decrypted_text)


Original Text: 98765432
Encrypted Text: c4dccc987f3dd312
Decrypted Text: 98765432


In [ ]:
!pip install cryptography==2.8
!pip install pycryptodome==3.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.4 MB/s eta 0:00:00


In [ ]:
import base64
import hashlib
import binascii
from Crypto import Random
from Crypto.Cipher import AES
from flask import current_app as app

from sqlalchemy import String
from sqlalchemy import TypeDecorator

BS = 16

pad = lambda s: s + (BS - len(s) % BS) * chr(BS - len(s) % BS)
unpad = lambda s: s[0:-s[-1]].decode()
class AESCipher:

    def __init__(self, key):
        self.key = key.encode('utf8')

    def encrypt(self, raw):
        raw = pad(raw)
        iv = Random.new().read(AES.block_size)
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return base64.b64encode(iv + cipher.encrypt(raw.encode('utf8'))).decode()

    def decrypt(self, enc):
        enc = base64.b64decode(enc)
        iv = enc[:16]
        cipher = AES.new(self.key, AES.MODE_CBC, iv)
        return unpad(cipher.decrypt(enc[16:]))

class SymEncryption:

    def __init__(self, key, nonce):
        self.key = key.encode('utf8')
        self.nonce = nonce.encode('utf8')

    def sym_encrypt(self, data):
        cipher = AES.new(self.key, AES.MODE_EAX, nonce=self.nonce)
        data = data + (" " * (16 - (len(data) % 16)))
        return cipher.encrypt(data.encode("utf-8")).hex()

    def sym_decrypt(self, data):
        cipher = AES.new(self.key, AES.MODE_EAX, nonce=self.nonce)
        data = data.encode('utf-8')
        return cipher.decrypt(binascii.unhexlify(data)).decode("utf-8").rstrip()


class EncryptedValue(TypeDecorator):
    impl = String

    def process_bind_param(self, value, dialect):
        if type(value) == list:
            return value
        ec_key = 'SYMENCKE'
        ec_nonce = 'SYMCRYPTON'
        sym = SymEncryption(ec_key, ec_nonce)
        return sym.sym_encrypt(value)

    def process_result_value(self, value, dialect):
        if value:
            if len(value) < 32:
                return value
            else:
                try:
                    ec_key = 'SYMENCKE'
                    ec_nonce = 'SYMCRYPTON'
                    sym = SymEncryption(ec_key, ec_nonce)
                    return sym.sym_decrypt(value)
                except:
                    return value
        else:
            return ''

class SHA:
    @staticmethod
    def encrypt(hash_string):
        sha_signature = hashlib.sha256(hash_string.encode()).hexdigest()
        return sha_signature


In [ ]:
original_value = "1000"

crypto_key = "00112233445566778899aabbccddeeff"
crypto_nonce = "SYMCRYPTON"

# Encryption using AESCipher
aes_cipher = AESCipher(crypto_key)
encrypted_aes = aes_cipher.encrypt(original_value)
print(f"Encrypted using AESCipher: {encrypted_aes}")

# Decryption using AESCipher
decrypted_aes = aes_cipher.decrypt(encrypted_aes)
print(f"Decrypted using AESCipher: {decrypted_aes}")

# Encryption using SymEncryption
sym_cipher = SymEncryption(crypto_key, crypto_nonce)
encrypted_sym = sym_cipher.sym_encrypt(original_value)
print(f"Encrypted using SymEncryption: {encrypted_sym}")

# Decryption using SymEncryption
decrypted_sym = sym_cipher.sym_decrypt(encrypted_sym)
print(f"Decrypted using SymEncryption: {decrypted_sym}")


Encrypted using AESCipher: PHCocVWUSH7tgqEX8J6m9IWMem3/liFG5iAdEO/2re8=
Decrypted using AESCipher: 1000
Encrypted using SymEncryption: 3e7605e5527ecc45f1f04a98e69f68ca
Decrypted using SymEncryption: 1000


In [ ]:
!pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.2 MB/s eta 0:00:00


In [ ]:
from faker import Faker
fake = Faker()

In [ ]:
print("Current VIN:", fake.uuid4())
print("Current Make:", fake.company())
print("Current Model:", fake.word())
print("Current Year:", fake.year())
print("Current Mileage:", fake.random_int(min=1000, max=50000))
print("Current Condition:", fake.word())
print("Current Value:", fake.random_int(min=10000, max=50000))

print("Interest VIN:", fake.uuid4())
print("Interest Make:", fake.company())
print("Interest Model:", fake.word())
print("Interest Trim:", fake.word())
print("Interest Stock:", fake.uuid4())
print("Interest Year:", fake.year())
print("Interest Price:", fake.random_int(min=10000, max=50000))
print("Interest Status: Available")
print("Interest Mileage:", fake.random_int(min=1000, max=50000))
print("Interest Condition:", fake.word())
print("Interest Listing URL:", fake.url())


Current VIN: fd9ee735-330f-45be-8e15-8ff03d1be01c
Current Make: Rice and Sons
Current Model: at
Current Year: 1998
Current Mileage: 6765
Current Condition: move
Current Value: 42028
Interest VIN: 1f4b4e9f-f9e2-4b15-bef2-2a06a8c8613d
Interest Make: Martin PLC
Interest Model: lose
Interest Trim: anyone
Interest Stock: c46f6265-8795-4397-8051-78d7f950f81a
Interest Year: 1992
Interest Price: 48780
Interest Status: Available
Interest Mileage: 11900
Interest Condition: eye
Interest Listing URL: http://cowan-mccall.com/
